<a href="https://colab.research.google.com/github/SusanLL/ClassifyCustomerReviews/blob/main/Machine_Translation_of_%E2%80%9CPride_and_Prejudice%E2%80%9D_Using_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers requests bs4 torch

import requests
from bs4 import BeautifulSoup
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead
import torch

In [ ]:
# Chapther II
def fetch_book_text(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    book_text = soup.get_text()

    # Optional: Trim the book text to remove preamble or end notes
    start_text = "R. BENNET was am"
    end_text = "him to dinner."
    start_idx = book_text.find(start_text)
    end_idx = book_text.rfind(end_text)
    main_content = book_text[start_idx:end_idx].strip()

    return main_content

In [ ]:
url = "https://www.gutenberg.org/cache/epub/1342/pg1342-images.html"
book_text = fetch_book_text(url)
print(book_text[:3000])

R. BENNET was among the earliest of those who waited on Mr. Bingley. He
had always intended to visit him, though to the last always assuring his
wife that he should not go; and till the evening after the visit was
paid she had no knowledge of it. It was then disclosed in the following
manner. Observing his second daughter employed in trimming a hat, he
suddenly addressed her with,—
“I hope Mr. Bingley will like it, Lizzy.”
“We are not in a way to know what Mr. Bingley likes,” said her mother,
resentfully, “since we are not to visit.{7}”
“But you forget, mamma,” said Elizabeth, “that we shall meet him at the
assemblies, and that Mrs. Long has promised to introduce him.”
“I do not believe Mrs. Long will do any such thing. She has two nieces
of her own. She is a selfish, hypocritical woman, and I have no opinion
of her.”
“No more have I,” said Mr. Bennet; “and I am glad to find that you do
not depend on her serving you.”
Mrs. Bennet deigned not to make any reply; but, unable to contain
he

In [ ]:
text_to_translate = book_text[:1000]

Let's translate the our text to Spanish:

In [ ]:
translator_es = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Translate the selected portion of the book text into Spanish
translated_text_es = translator_es(text_to_translate[:1000])[0]['translation_text']

In [ ]:
translated_text_es[:1000]

'R. BENNET era uno de los primeros que esperaban al Sr. Bingley. Siempre había tenido la intención de visitarlo, aunque hasta el último siempre asegurando a su esposa que no debía ir; y hasta la noche después de la visita se le pagó que no tenía conocimiento de ella. Entonces se reveló de la siguiente manera. Observando a su segunda hija empleada en recortar un sombrero, de repente se dirigió a ella con, - "Espero que al Sr. Bingley le guste, Lizzy." "No estamos en una manera de saber lo que le gusta al Sr. Bingley", dijo su madre, con resentimiento, "ya que no vamos a visitar. {7}" "Pero usted se olvida, mamá", dijo Elizabeth, "que nos reuniremos con él en las asambleas, y que la Sra. Long ha prometido presentarlo." "No creo que la Sra. Long hará tal cosa. Ella tiene dos sobrinas propias. Ella es una mujer egoísta, hipócrita, y no tengo opinión de ella."'

In [ ]:
def calculate_perplexity(text, model_name):
    model = AutoModelWithLMHead.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        loss = model(**inputs, labels=inputs["input_ids"]).loss
    return torch.exp(loss).item()

In [ ]:
spanish_perplexity_model = "dccuchile/bert-base-spanish-wwm-cased"
spanish_perplexity = calculate_perplexity(translated_text_es, spanish_perplexity_model)

In [ ]:
print("Perplexity for Spanish Translation:", spanish_perplexity)

Perplexity for Spanish Translation: 1.7127097845077515


Perplexity ≈ 1: Indicates very high fluency and coherence, as it means the model finds the text almost perfectly aligned with standard language usage.

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
reference_text_es = "El	señor	Bennet	fue	uno	de	los	primeros	en	presentar	sus	respetos	al	señor Bingley.	Siempre	tuvo	la	intención	de	visitarlo,	aunque,	al	final,	siempre	le aseguraba	a	su	esposa	que	no	lo	haría;	y	hasta	la	tarde	después	de	su	visita,	su mujer	no	se	enteró	de	nada.	La	cosa	se	llegó	a	saber	de	la	siguiente	manera: observando	el	señor	Bennet	cómo	su	hija	se	colocaba	un	sombrero,	dijo:––Espero	que	al	señor	Bingley	le	guste,	Lizzy.––¿Cómo	podemos	saber	qué	le	gusta	al	señor	Bingley	––dijo	su	esposa resentida––	si	todavía	no	hemos	ido	a	visitarlo?––Olvidas,	mamá	––dijo	Elizabeth––	que	lo	veremos	en	las	fiestas,	y	que la	señora	Long	ha	prometido	presentárnoslo.––No	creo	que	la	señora	Long	haga	semejante	cosa.	Ella	tiene	dos	sobrinas en	quienes	pensar;	es	egoísta	e	hipócrita	y	no	merece	mi	confianza.––Ni	la	mía	tampoco	––dijo	el	señor	Bennet––	y	me	alegro	de	saber	que	no dependes	de	sus	servicios.	La	señora	Bennet	no	se	dignó	contestar;	pero incapaz	de	contenerse	empezó	a	reprender	a	una	de	sus	hijas.––¡Por	el	amor	de	Dios,	Kitty	no	sigas	tosiendo	así!	Ten	compasión	de	mis nervios.	Me	los	estás	destrozando.––Kitty	no	es	nada	discreta	tosiendo	––dijo	su	padre––.	Siempre	lo	hace	en momento	inoportuno.	––A	mí	no	me	divierte	toser	––replicó	Kitty	quejándose.––¿Cuándo	es	tu	próximo	baile,	Lizzy?––De	mañana	en	quince	días.––Sí,	así	es	––exclamó	la	madre––.	Y	la	señora	Long	no	volverá	hasta	un día	antes;	así	que	le	será	imposible	presentarnos	al	señor	Bingley,	porque todavía	no	le	conocerá.––Entonces,	señora	Bennet,	puedes	tomarle	la	delantera	a	tu	amiga	y presentárselo	tú	a	ella.	––Imposible,	señor	Bennet,	imposible,	cuando	yotampoco	le	conozco.	¿Por	qué	te	burlas?––Celebro	tu	discreción.	Una	amistad	de	quince	días	es	verdaderamente muy	poco.	En	realidad,	al	cabo	de	sólo	dos	semanas	no	se	puede	saber	muy bien	qué	clase	de	hombre	es.	Pero	si	no	nos	arriesgamos	nosotros,	lo	harán otros.	Al	fin	y	al	cabo,	la	señora	Long	y	sus	sobrinas	pueden	esperar	a	que	se les	presente	su	oportunidad;	pero,	no	obstante,	como	creerá	que	es	un	acto	de delicadeza	por	su	parte	el	declinar	la	atención,	seré	yo	el	que	os	lo	presente. Las	muchachas	miraron	a	su	padre	fijamente.	La	señora	Bennet	se	limitó	a decir:	––¡Tonterías,	tonterías!––¿Qué	significa	esa	enfática	exclamación?	––preguntó	el	señor	Bennet––. ¿Consideras	las	fórmulas	de	presentación	como	tonterías,	con	la	importancia que	tienen?	No	estoy	de	acuerdo	contigo	en	eso.	¿Qué	dices	tú,	Mary?	Que	yo sé	que	eres	una	joven	muy	reflexiva,	y	que	lees	grandes	libros	y	los	resumes. Mary	quiso	decir	algo	sensato,	pero	no	supo	cómo.––Mientras	Mary	aclara	sus	ideas	––continuó	él––,	volvamos	al	señor Bingley.	––¡Estoy	harta	del	señor	Bingley!	––gritó	su	esposa."

In [ ]:
reference_tokens = [reference_text_es.split()]
generated_tokens = translated_text_es.split()

In [ ]:
smoothie = SmoothingFunction().method4  # Smoothing function for small texts
bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothie)

print("BLEU Score for Spanish Translation:", bleu_score)

BLEU Score for Spanish Translation: 0.02335579328371668


In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer for ROUGE-1, ROUGE-2, and ROUGE-L
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores (reference and generated translations)
rouge_scores = scorer.score(reference_text_es, translated_text_es)

# Display results
print("ROUGE-1 Score:", rouge_scores['rouge1'].fmeasure)
print("ROUGE-2 Score:", rouge_scores['rouge2'].fmeasure)
print("ROUGE-L Score:", rouge_scores['rougeL'].fmeasure)

ROUGE-1 Score: 0.3538011695906432
ROUGE-2 Score: 0.17008797653958946
ROUGE-L Score: 0.260233918128655
